In [73]:
## Liberías

# Clásicas de análisis de datos
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

# Librerías de ML
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Métricas de modelos
from sklearn.metrics import confusion_matrix

In [74]:
# Dataset Original
df = pd.read_csv('./Dataset/Anime2.csv', index_col=0)
df.head(10)

,Expertos,Capitulos,Animes,Edad,Mujer,CantCaps,Scoremyanimelist,NEscenasCombate,NescenasEmocionantes,CalidadAnimacion,DuracionCapituloMin,RatingTV,Eleccion
1.1.1,1,1,1,42,1,575,8.0,3,2,4,41,13,0
1.1.2,1,1,2,42,1,720,6.5,4,1,7,54,27,1
1.1.3,1,1,3,42,1,148,10.0,1,0,2,30,20,0
1.1.4,1,1,4,42,1,113,4.0,2,1,8,53,15,0
1.2.1,1,2,1,42,1,575,8.0,4,1,7,59,25,0
1.2.2,1,2,2,42,1,720,6.5,4,3,6,32,19,0
1.2.3,1,2,3,42,1,148,10.0,2,3,3,44,38,1
1.2.4,1,2,4,42,1,113,4.0,4,2,4,35,37,0
1.3.1,1,3,1,42,1,575,8.0,2,1,2,45,29,0
1.3.2,1,3,2,42,1,720,6.5,2,2,4,34,32,1


In [75]:
df['RatingTV'].median()

22.5

In [76]:
# Cambiamos el nombre de algunas columnas para facilitar la vida

# Atributos a agrupar en una sola fila por eleccion 
cambio_nombre = {'NEscenasCombate':'Combat', 'NescenasEmocionantes':'Emotion', 'CalidadAnimacion':'Qual', 'DuracionCapituloMin':'Duration', 'RatingTV':'Rating', 'CantCaps':'Caps', 'Scoremyanimelist':'Score'}

# Cambio de nombre
df.rename(cambio_nombre, axis=1, inplace=True)

df.head()

,Expertos,Capitulos,Animes,Edad,Mujer,Caps,Score,Combat,Emotion,Qual,Duration,Rating,Eleccion
1.1.1,1,1,1,42,1,575,8.0,3,2,4,41,13,0
1.1.2,1,1,2,42,1,720,6.5,4,1,7,54,27,1
1.1.3,1,1,3,42,1,148,10.0,1,0,2,30,20,0
1.1.4,1,1,4,42,1,113,4.0,2,1,8,53,15,0
1.2.1,1,2,1,42,1,575,8.0,4,1,7,59,25,0


### Homogéneo

Formato de clases (Previo a Wide)

In [77]:
# Dropeamos la columnas que no nos sirven para el análisis homogéneo
# Expertos = indiv, capitulos = tiempo, anime = producto
df_homogeneo = df.copy() #.drop(columns=['CantCaps', 'Scoremyanimelist'])

# La elección la pasamos a categorías
df_homogeneo['Eleccion'] = df_homogeneo['Eleccion'] * df_homogeneo['Animes']

# Eliminamos anime que ya es parte de Eleccion
df_homogeneo.drop(columns=['Animes', 'Edad', 'Rating', 'Score', 'Caps'], inplace=True)

df_homogeneo.head(12)

,Expertos,Capitulos,Mujer,Combat,Emotion,Qual,Duration,Eleccion
1.1.1,1,1,1,3,2,4,41,0
1.1.2,1,1,1,4,1,7,54,2
1.1.3,1,1,1,1,0,2,30,0
1.1.4,1,1,1,2,1,8,53,0
1.2.1,1,2,1,4,1,7,59,0
1.2.2,1,2,1,4,3,6,32,0
1.2.3,1,2,1,2,3,3,44,3
1.2.4,1,2,1,4,2,4,35,0
1.3.1,1,3,1,2,1,2,45,0
1.3.2,1,3,1,2,2,4,34,2


In [78]:
# Agrupamos para juntar la información de una combinación (i,j,t) en una sola (juntar los 4 animé en una sola fila y con la elección como categórica)

# Tupla
tupla = ['Expertos', 'Capitulos'] # El animé es parte de la eleecion

# Atributos a agrupar en una sola fila por eleccion 
atributos_eleccion = ['Combat', 'Emotion', 'Qual', 'Duration']

# Animes
animes = ['hunterX', 'dragonball', 'naruto', 'bnha']

# Grop by según tupla
gb = df_homogeneo.groupby(tupla)

# Creamos una copia donde vamos a guardar el resultado final
df_homogeneo_agrupado = df_homogeneo.copy()

# Eliminamos las filas donde no hay eleccion
df_homogeneo_agrupado = df_homogeneo_agrupado[df_homogeneo_agrupado['Eleccion'] > 0].reset_index()

# Eliminamos las columnas que agrupamos
df_homogeneo_agrupado.drop(columns=atributos_eleccion, inplace=True)

# Recorro cada anime y veo si calza con el original
for i in range(4):

    # Recorro cada atributo a renombrar
    for atr in atributos_eleccion:

        # Agrego el atributo renombrado al dataframe
        df_homogeneo_agrupado[f'{atr}.{animes[i]}'] = 0
        df_homogeneo_agrupado[f'{atr}.{animes[i]}'] = gb.nth(i).reset_index()[atr]

    # Renombramos también la columna de Eleccion
    df_homogeneo_agrupado.loc[ df_homogeneo_agrupado['Eleccion'] == i+1, 'Eleccion' ] = animes[i]

# Eliminamos la columna de indice vieja y los indices de la tupla
df_homogeneo_agrupado.drop(columns=['index', 'Expertos', 'Capitulos'], inplace=True)

# # Renombramos la columna de elección
# for i in range
# df_homogeneo_agrupado['Eleccion']

# Aliminamos las 
df_homogeneo_agrupado.head(12)

,Mujer,Eleccion,Combat.hunterX,Emotion.hunterX,Qual.hunterX,Duration.hunterX,Combat.dragonball,Emotion.dragonball,Qual.dragonball,Duration.dragonball,Combat.naruto,Emotion.naruto,Qual.naruto,Duration.naruto,Combat.bnha,Emotion.bnha,Qual.bnha,Duration.bnha
0,1,dragonball,3,2,4,41,4,1,7,54,1,0,2,30,2,1,8,53
1,1,naruto,4,1,7,59,4,3,6,32,2,3,3,44,4,2,4,35
2,1,dragonball,2,1,2,45,2,2,4,34,1,2,1,46,2,1,6,48
3,1,dragonball,3,1,7,39,4,2,6,58,3,0,8,32,2,2,8,54
4,1,dragonball,1,1,6,46,4,2,7,39,3,0,9,56,1,2,2,50
5,1,bnha,4,2,6,55,3,1,2,58,3,0,4,55,0,0,9,59
6,1,dragonball,4,2,6,31,0,1,3,57,0,3,6,44,2,0,1,45
7,1,dragonball,2,0,9,39,2,0,8,31,4,1,6,35,3,3,4,34
8,1,dragonball,4,2,2,51,1,2,8,44,1,3,6,53,2,3,5,33
9,1,dragonball,2,1,4,46,2,3,4,56,3,1,7,30,1,2,9,35


In [79]:
# Tasa de Emotion
df_homogeneo_agrupado[[f'Emotion.{anime}' for anime in animes]] = 30*df_homogeneo_agrupado[[f'Emotion.{anime}' for anime in animes]].to_numpy() / df_homogeneo_agrupado[[f'Duration.{anime}' for anime in animes]].to_numpy()

# Tasa de Combat
df_homogeneo_agrupado[[f'Combat.{anime}' for anime in animes]] = 30*df_homogeneo_agrupado[[f'Combat.{anime}' for anime in animes]].to_numpy() / df_homogeneo_agrupado[[f'Duration.{anime}' for anime in animes]].to_numpy()

df_homogeneo_agrupado = df_homogeneo_agrupado.drop(columns=[f'Duration.{anime}' for anime in animes])
df_homogeneo_agrupado.head()

,Mujer,Eleccion,Combat.hunterX,Emotion.hunterX,Qual.hunterX,Combat.dragonball,Emotion.dragonball,Qual.dragonball,Combat.naruto,Emotion.naruto,Qual.naruto,Combat.bnha,Emotion.bnha,Qual.bnha
0,1,dragonball,2.195122,1.463415,4,2.222222,0.555556,7,1.000000,0.000000,2,1.132075,0.566038,8
1,1,naruto,2.033898,0.508475,7,3.750000,2.812500,6,1.363636,2.045455,3,3.428571,1.714286,4
2,1,dragonball,1.333333,0.666667,2,1.764706,1.764706,4,0.652174,1.304348,1,1.250000,0.625000,6
3,1,dragonball,2.307692,0.769231,7,2.068966,1.034483,6,2.812500,0.000000,8,1.111111,1.111111,8
4,1,dragonball,0.652174,0.652174,6,3.076923,1.538462,7,1.607143,0.000000,9,0.600000,1.200000,2


In [80]:
df_homogeneo_agrupado

,Mujer,Eleccion,Combat.hunterX,Emotion.hunterX,Qual.hunterX,Combat.dragonball,Emotion.dragonball,Qual.dragonball,Combat.naruto,Emotion.naruto,Qual.naruto,Combat.bnha,Emotion.bnha,Qual.bnha
0,1,dragonball,2.195122,1.463415,4,2.222222,0.555556,7,1.000000,0.000000,2,1.132075,0.566038,8
1,1,naruto,2.033898,0.508475,7,3.750000,2.812500,6,1.363636,2.045455,3,3.428571,1.714286,4
2,1,dragonball,1.333333,0.666667,2,1.764706,1.764706,4,0.652174,1.304348,1,1.250000,0.625000,6
3,1,dragonball,2.307692,0.769231,7,2.068966,1.034483,6,2.812500,0.000000,8,1.111111,1.111111,8
4,1,dragonball,0.652174,0.652174,6,3.076923,1.538462,7,1.607143,0.000000,9,0.600000,1.200000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,0,hunterX,1.538462,0.000000,9,1.935484,0.000000,8,3.428571,0.857143,6,2.647059,2.647059,4
11996,0,hunterX,2.352941,1.176471,2,0.681818,1.363636,8,0.566038,1.698113,6,1.818182,2.727273,5
11997,0,hunterX,1.304348,0.652174,4,1.071429,1.607143,4,3.000000,1.000000,7,0.857143,1.714286,9
11998,0,hunterX,2.068966,0.517241,9,0.000000,0.967742,5,0.638298,1.914894,7,0.000000,0.000000,7


In [81]:
# Guardemos el dataset para correr los modelos en R
df_homogeneo_agrupado.to_csv('./Dataset/Anime2_Logit.csv', index=False)